In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image


input_folders = {
    0: "/kaggle/input/brain-tumors-256x256/Data/glioma_tumor",
    1: "/kaggle/input/brain-tumors-256x256/Data/meningioma_tumor",
    2: "/kaggle/input/brain-tumors-256x256/Data/normal",
    3: "/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor"
}


output_csv_path = "/kaggle/working/combined_dataset.csv"
data_rows = []


for label, folder_path in input_folders.items():
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".jpg"):
            file_path = os.path.join(folder_path, file_name)

           
            image = Image.open(file_path).convert("L")

           
            image = image.resize((256, 256))

          
            image_array = np.array(image).flatten()

           
            row = [label] + image_array.tolist()
            data_rows.append(row)


columns = ["label"] + [f"pixel{i}" for i in range(256 * 256)]
dataset = pd.DataFrame(data_rows, columns=columns)


dataset.to_csv(output_csv_path, index=False)

print(f"Dataset saved to {output_csv_path}")




In [ ]:
data = pd.read_csv('/kaggle/working/combined_dataset.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape



In [ ]:
def init_params():
    W1 = np.random.rand(16384, 65536) - 0.5
    b1 = np.random.rand(16384, 1) - 0.5
    W2 = np.random.rand(4096, 16384) - 0.5
    b2 = np.random.rand(4096, 1) - 0.5
    W3 = np.random.rand(1024, 4096) - 0.5
    b3 = np.random.rand(1024, 1) - 0.5
    W4 = np.random.rand(4, 1024) - 0.5
    b4 = np.random.rand(4, 1) - 0.5
    return W1, b1, W2, b2, W3, b3, W4, b4

In [ ]:
# Activation Functions

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  
    return expZ / np.sum(expZ, axis=0, keepdims=True)

In [ ]:
def forward_prop(X, W1, b1, W2, b2, W3, b3, W4, b4):
    # L1
    Z1 = np.dot(W1, X) + b1
    A1 = ReLU(Z1)
    # L2
    Z2 = np.dot(W2, A1) + b2
    A2 = ReLU(Z2)
    # L3
    Z3 = np.dot(W3, A2) + b3
    A3 = ReLU(Z3)
    # L4/Output
    Z4 = np.dot(W4, A3) + b4
    A4 = softmax(Z4)
    
    return Z1, A1, Z2, A2, Z3, A3, Z4, A4